In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from support_code.model import DQN
from support_code.env import TradingEnv
from support_code.functions import fetch_data, buy_and_hold, unpickle_model, corr

In [ ]:
warnings.simplefilter('ignore')

Load in stock price history for one company:

In [ ]:
stock = 'GOOG'
df = fetch_data(stock.lower()+'_df')
print(f'Stock: {stock}')

---

Initialize the environment for testing:

In [ ]:
starting_balance = 1_000_000

env = TradingEnv(df, balance_init=starting_balance, training=False)

env.verbose=1
env.reset();

Save global variables for the price history, buy and hold performance, and testing date range:

In [ ]:
PRICE_HISTORY = df['close'].loc[env.train_test_split:].values[:-1]
LONG_STRATEGY = buy_and_hold(balance_init=env.balance_init, back_prices=PRICE_HISTORY, fee=env.fee)
DATE_RANGE = pd.to_datetime(df['date'].loc[env.train_test_split:].values)[:-1]

Simulate the performance of a random agent:

In [ ]:
done = False
obs = env.reset()
start = env.current_step

model_performance = []
actions = []

while not done:
    
    model_performance.append(env.net_worth)
        
    action = np.random.randint(env.action_space.n)
    obs, rewards, done, info = env.step(action)

    actions.append(env._actions[action])
    
end = env.current_step

In [ ]:
date_range_ = pd.to_datetime(df['date'].values[start:end])

Plot the performance of the agent:

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax1.plot(DATE_RANGE, LONG_STRATEGY, 'b--', alpha=0.5, label='Buy and Hold')
ax1.plot(date_range_, model_performance, 'r-', label='Backtesting Model')
ax1.axhline(env.balance_init, alpha=0.1, color='blue')
ax1.set_ylabel('Portfolio Value (M)')
ax1.set_title('Random Actions')
ax1.legend()

ax2.plot(DATE_RANGE, PRICE_HISTORY, 'b-', label=stock)
ax2.set_ylabel('Stock Value')
ax2.legend()

fig.tight_layout()
fig.savefig(f'./assets/{stock}_performance_rand.jpg');
fig.show();

Plot the actions taken by the agent:

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(actions, 'b-', alpha=0.6667)
plt.plot(actions, 'r.')

plt.title('Action Trace')
plt.xlabel('Step')
plt.ylabel('Portion of Portfolio Bought/Sold')

plt.tight_layout()
plt.savefig(f'./assets/{stock}_actions_rand.png')
plt.show();

Find the correlation between actions and price history:

In [ ]:
print(corr(actions, PRICE_HISTORY[:len(actions)]))

---

Unpickle the model to be tested:

In [ ]:
dqn_solver = unpickle_model(DQN, path='./models/model_info_sp500_1')
dqn_solver.alpha = 0.01
dqn_solver.alpha_min = 0.01

Simulate the agent's performance:

In [ ]:
model_performance = []
actions = []
    
state = env.reset()
start = env.current_step
done = False    

while not done:

    model_performance.append(env.net_worth)  

    action = dqn_solver.act(state)
    next_state, reward, done, info = env.step(action)

    dqn_solver.remember(state, action, reward, next_state, done)
    state = next_state

    actions.append(action)
    #dqn_solver.replay();  

In [ ]:
date_range_ = pd.to_datetime(df['date'].values[env.train_test_split:env.train_test_split+len(model_performance)])

Plot the performance of the agent:

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ax1.plot(DATE_RANGE, LONG_STRATEGY, 'b--', alpha=0.7, label='Buy and Hold')
ax1.plot(date_range_, model_performance, 'r-', label='Backtesting Model')
ax1.axhline(env.balance_init, alpha=0.1, color='blue')
ax1.set_ylabel('Portfolio Value (M)')
ax1.set_title('Intelligent/Trained Actions')
ax1.legend()

ax2.plot(DATE_RANGE, PRICE_HISTORY, 'b-', label=stock)
ax2.set_ylabel('Stock Value')
ax2.legend(loc='upper left')

fig.tight_layout()
fig.savefig(f'./assets/{stock}_performance_trained.jpg');
fig.show();

Plot the actions taken by the agent:

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(actions, 'b-', alpha=0.6667)
plt.plot(actions, 'r.')

plt.title('Action Trace')
plt.xlabel('Step')
plt.ylabel('Portion of Portfolio Bought/Sold')
plt.yticks(np.arange(len(env._actions)), env._actions)

plt.tight_layout()
plt.savefig(f'./assets/{stock}_actions_trained.png')
plt.show();

Find the correlation between actions and price history:

In [ ]:
print(corr(actions, PRICE_HISTORY[:len(actions)]))